In [1]:
import pandas as pd
import numpy as np
from data_preprocessor import DataPreprocessor
from sklearn.model_selection import train_test_split

# 1. ПОДГОТОВКА ДАННЫХ
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

# добавим искусственную колонку с пропусками (90% NaN), чтобы увидеть её удаление
df['Trash_Column'] = np.nan
df.loc[0:10, 'Trash_Column'] = 1.0

train, test = train_test_split(df, test_size=0.2, random_state=42)

# ЭТАП ДО
print("=== ДАННЫЕ ДО ОБРАБОТКИ ===")
print(f"Размер таблицы: {train.shape[0]} строк, {train.shape[1]} колонок")
print(f"Колонки: {train.columns.tolist()}")
display(train.head(10))
print("-" * 50)

# 2. ОБРАБОТКА
# оставляем только нужные колонки
cols_to_use = ['Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'Cabin']
train_subset = train[cols_to_use]

preprocessor = DataPreprocessor(train_subset)

# ЗДЕСЬ МОЖНО ИЗМЕНИТЬ ПАРАМЕТРЫ 
train_p = preprocessor.fit_transform(threshold=0.5, method='std', strategy='median')
test_p = preprocessor.transform(test)

# ЭТАП ПОСЛЕ
print("=== ДАННЫЕ ПОСЛЕ ОБРАБОТКИ ===")
print(f"Размер таблицы: {train_p.shape[0]} строк, {train_p.shape[1]} колонок")
display(train_p.head(10))
print("-" * 50)

# 3. ОТЧЕТ ОБ ИЗМЕНЕНИЯХ
print("=== ЧТО БЫЛО СДЕЛАНО ===")

# удаленные колонки
removed = list(set(df.columns) - set(preprocessor.params['cols_to_keep']))
print(f"1. Удалено колонок ({len(removed)} шт.): {removed}")

# колонки, в которых заполняли пропуски
imputed = [col for col, val in preprocessor.params['impute_values'].items() if train[col].isnull().any()]
print(f"2. Заполнены пропуски в ({len(imputed)} шт.): {imputed}")

# категории, которые превратились в One-Hot
encoded = preprocessor.params['cat_columns']
print(f"3. Закодированы категории ({len(encoded)} шт.): {encoded}")

# числовые признаки, которые нормализовали
normalized = list(preprocessor.params['scaling_stats'].keys())
print(f"4. Нормализованы числа ({len(normalized)} шт.) методом '{preprocessor.params['norm_method']}': {normalized}")

print(f"\nИтоговое количество признаков для модели: {len(preprocessor.params['final_columns'])}")

=== ДАННЫЕ ДО ОБРАБОТКИ ===
Размер таблицы: 712 строк, 13 колонок
Колонки: ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Trash_Column']


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Trash_Column
331,332,0,1,"Partner, Mr. Austen",male,45.5,0,0,113043,28.5000,C124,S,NaN
733,734,0,2,"Berriman, Mr. William John",male,23.0,0,0,28425,13.0000,NaN,S,NaN
382,383,0,3,"Tikkanen, Mr. Juho",male,32.0,0,0,STON/O 2. 3101293,7.9250,NaN,S,NaN
704,705,0,3,"Hansen, Mr. Henrik Juul",male,26.0,1,0,350025,7.8542,NaN,S,NaN
813,814,0,3,"Andersson, Miss. Ebba Iris Alfrida",female,6.0,4,2,347082,31.2750,NaN,S,NaN
118,119,0,1,"Baxter, Mr. Quigg Edmond",male,24.0,0,1,PC 17558,247.5208,B58 B60,C,NaN
536,537,0,1,"Butt, Major. Archibald Willingham",male,45.0,0,0,113050,26.5500,B38,S,NaN
361,362,0,2,"del Carlo, Mr. Sebastiano",male,29.0,1,0,SC/PARIS 2167,27.7208,NaN,C,NaN
29,30,0,3,"Todoroff, Mr. Lalio",male,NaN,0,0,349216,7.8958,NaN,S,NaN
55,56,1,1,"Woolner, Mr. Hugh",male,NaN,0,0,19947,35.5000,C52,S,NaN


--------------------------------------------------
=== ДАННЫЕ ПОСЛЕ ОБРАБОТКИ ===
Размер таблицы: 712 строк, 7 колонок


,Survived,Pclass,Age,Fare,Sex_male,Embarked_Q,Embarked_S
331,0,-1.613002,1.252760,-0.078628,1,0,1
733,0,-0.400270,-0.476948,-0.376880,1,0,1
382,0,0.812463,0.214935,-0.474533,1,0,1
704,0,0.812463,-0.246320,-0.475896,1,0,1
813,0,0.812463,-1.783839,-0.025232,0,0,1
118,0,-1.613002,-0.400072,4.135780,1,0,0
536,0,-1.613002,1.214322,-0.116150,1,0,1
361,0,-0.400270,-0.015693,-0.093622,1,0,0
29,0,0.812463,-0.092569,-0.475095,1,0,1
55,1,-1.613002,-0.092569,0.056066,1,0,1


--------------------------------------------------
=== ЧТО БЫЛО СДЕЛАНО ===
1. Удалено колонок (7 шт.): ['Parch', 'PassengerId', 'Ticket', 'SibSp', 'Name', 'Cabin', 'Trash_Column']
2. Заполнены пропуски в (2 шт.): ['Age', 'Embarked']
3. Закодированы категории (2 шт.): ['Sex', 'Embarked']
4. Нормализованы числа (3 шт.) методом 'std': ['Pclass', 'Age', 'Fare']

Итоговое количество признаков для модели: 7
